In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def get_laplacian(requires_grad=False):    
    hood = [[ 1.,   4., 1.],
            [ 4., -20., 4.],
            [ 1.,   4., 1.],]

    zeros = [[0., 0., 0.],
             [0., 0., 0.],
             [0., 0., 0.],]

    weights = torch.tensor([
        [hood, zeros, zeros],
        [zeros, hood, zeros],
        [zeros, zeros, hood],
    ])
    res = nn.Conv2d(3, 3, 3, stride=1, padding=1, 
        bias=False, padding_mode='replicate')
    # nn.Conv2d(3, 3, self.kernel_size, stride=1, padding=0, bias=None, groups=3)
    
    res.requires_grad_(requires_grad)
    res.weight.data = weights
    return res

#laplacian = 

In [3]:
from collections import OrderedDict

class DiscreteLaplacian(nn.Module):
    def __init__(self):
        super(DiscreteLaplacian, self).__init__()
        
        self.seq = nn.Sequential(OrderedDict([
          #('padd', nn.ReflectionPad2d(1)),
          ('conv', nn.Conv2d(3, 3, 3, stride=1, padding=0, 
                             bias=False, groups=3)),
        ]))
        self.seq.requires_grad_(False)
        self.weights_init()

    def forward(self, x):
        return self.seq(x)#.sum()

    def weights_init(self):
        w = torch.tensor([[ 1.,   4., 1.],
                          [ 4., -20., 4.],
                          [ 1.,   4., 1.],])
        for _, f in self.named_parameters():           
            f.data.copy_(w)

dl = DiscreteLaplacian()            
dl

DiscreteLaplacian(
  (seq): Sequential(
    (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), groups=3, bias=False)
  )
)

In [4]:
dl.seq.conv.weight.data

tensor([[[[  1.,   4.,   1.],
          [  4., -20.,   4.],
          [  1.,   4.,   1.]]],


        [[[  1.,   4.,   1.],
          [  4., -20.,   4.],
          [  1.,   4.,   1.]]],


        [[[  1.,   4.,   1.],
          [  4., -20.,   4.],
          [  1.,   4.,   1.]]]])

In [7]:
dl(torch.randn(1, 3, 8, 8) * 0.01).norm()

tensor(2.2430)